In [1]:
import pandas as pd

In [2]:
user_table=pd.read_csv("userTable.csv")
vistorlog=pd.read_csv("VisitorLogsData.csv")
submission=pd.read_csv("sample_submission_M7Vpb9f.csv")

In [3]:
user_table.head()

,UserID,Signup Date,User Segment
0,U133159,2018-04-14 07:01:16.202607+00:00,C
1,U129368,2017-12-02 09:38:41.584270+00:00,B
2,U109654,2013-03-19 11:38:55+00:00,B
3,U108998,2018-01-18 08:29:51.627954+00:00,C
4,U131393,2018-03-27 08:05:28.806800+00:00,B


In [4]:
vistorlog.head()

,webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country
0,WI10000050298,2018-05-07 04:28:45.970,pr100631,NaN,NaN,Chrome Mobile,Android,Chennai,India
1,WI10000025922,2018-05-13 07:26:04.964,pr100707,NaN,NaN,Chrome,Windows,NaN,Taiwan
2,WI100000204522,2018-05-11 11:43:42.832,pr100030,NaN,click,Chrome,windows,Gurgaon,India
3,WI10000011974,2018-05-13 15:20:23.436,Pr100192,NaN,CLICK,Chrome,Windows,,
4,WI100000441953,2018-05-08 20:44:25.238,Pr100762,NaN,click,Chrome,mac os x,Iselin,United States


In [5]:
submission.head()

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
0,U100002,1,1,1,P12345,OS12345,P12345,1,1
1,U100003,1,1,1,P12345,OS12345,P12345,1,1
2,U100004,1,1,1,P12345,OS12345,P12345,1,1
3,U100005,1,1,1,P12345,OS12345,P12345,1,1
4,U100006,1,1,1,P12345,OS12345,P12345,1,1


In [6]:
data=vistorlog.merge(user_table,on='UserID',how='left') 

In [7]:
data['VisitDateTime']=data['VisitDateTime'].fillna('null')
data[['UserID','VisitDateTime']]=data[['UserID','VisitDateTime']].sort_values(by='UserID').replace('null',np.nan).ffill()

<IPython.core.display.Javascript object>

In [8]:
data['ProductID']=data['ProductID'].str.lower()
data['Activity']=data['Activity'].str.lower()
data['OS']=data['OS'].str.lower()

In [9]:
#data[data['UserID']=='U106593']

In [10]:
df=pd.DataFrame()
df['index']=submission['UserID']

In [11]:
import datetime
visit_date=[]
for i in data['VisitDateTime']:
    try:
        
        if type(int(i))==int:
            visit_date.append(datetime.datetime.fromtimestamp(int(i)/1000000000,datetime.timezone.utc).strftime('%Y-%m-%d %H:%M:%S.%f'))
    except ValueError :
        visit_date.append(i)


In [12]:
data['visit_date_time']=visit_date
data['visit_date_time'].fillna(np.nan,inplace=True)
data['visit_date_time']= pd.to_datetime(data['visit_date_time'])
data['visit_date_time'].max()

<IPython.core.display.Javascript object>

Timestamp('2018-05-27 23:59:59.576000')

In [13]:
days_visited=data['UserID'][data['visit_date_time']>'2018-05-21'].value_counts().to_frame(name='No_of_days_Visited_7_Days').reset_index(level=0)
df=df.merge(days_visited,on='index',how='left')

In [14]:
df.head()

,index,No_of_days_Visited_7_Days
0,U100002,NaN
1,U100003,3.0
2,U100004,1.0
3,U100005,2.0
4,U100006,1.0


In [15]:
#data[data['UserID']=='U100347']
#days_visited

In [16]:
sample=data[['UserID','ProductID','visit_date_time']][data['visit_date_time']>'2018-05-13']

Pro_view=sample.sort_values(by='visit_date_time',ascending=False).drop_duplicates(keep='first')
product_viewed=Pro_view['UserID'].value_counts().to_frame(name='No_Of_Products_Viewed_15_Days').reset_index(level=0)

In [17]:
df=df.merge(product_viewed,on='index',how='left')

In [18]:
vintage=data['UserID'].value_counts().to_frame(name='User_Vintage').reset_index(level=0)
df=df.merge(vintage,on='index',how='left')

In [19]:
#Test=sample.value_counts().to_frame(name='value').reset_index(level=1).reset_index(level=0)

In [20]:
sample=data[['UserID','ProductID','visit_date_time','Activity']][data['visit_date_time']>'2018-05-13']
ss=sample[sample['Activity']=='pageload'].sort_values(by='visit_date_time',ascending=False)

In [21]:
Most_Viewed_product_15_Days=ss[['UserID','ProductID']].value_counts().to_frame('value').reset_index(level=1).reset_index(level=0).drop_duplicates(subset='UserID',keep='first').drop(columns='value')

In [22]:
Most_Viewed_product_15_Days.columns=['index','Most_Viewed_product_15_Days']

In [23]:
#values=[]
#for ID in Test['UserID'].unique().tolist():
    #values.append(Test[Test['UserID']==ID].max().values[1])

In [24]:
#most_viewed=pd.DataFrame()
#most_viewed['index']=Test['UserID'].unique().tolist()

#most_viewed['Most_Viewed_product_15_Days']=values

In [25]:
df=df.merge(Most_Viewed_product_15_Days,on='index',how='left')


In [26]:
df['Most_Viewed_product_15_Days']=df['Most_Viewed_product_15_Days'].astype('str').replace('nan','Product101')

In [27]:
#data['UserID'].value_counts().to_frame(name='Vintage').reset_index(level=0)

In [28]:
most_activeOS=data[['UserID','OS']].value_counts().to_frame(name='value').reset_index(level=1).reset_index(level=0).drop_duplicates(subset='UserID',keep='first')

In [29]:
most_activeos=most_activeOS[['UserID','OS']]
most_activeos.columns=['index','OS']

In [30]:
df=df.merge(most_activeos,on='index',how='left')

In [31]:

Test3=data[['UserID','Activity','visit_date_time','ProductID']]

In [32]:
ss=Test3[Test3['Activity']=='pageload'].sort_values(by='visit_date_time',ascending=False).drop_duplicates(subset='UserID',keep='first')

In [33]:
ss

,UserID,Activity,visit_date_time,ProductID
2201536,U136965,pageload,2018-05-27 23:59:50.502,pr100431
5572200,U104305,pageload,2018-05-27 23:59:10.551,pr100102
2710310,U104304,pageload,2018-05-27 23:59:10.551,pr100275
2188125,U104296,pageload,2018-05-27 23:55:39.057,pr100390
1287857,U104309,pageload,2018-05-27 23:47:47.478,pr100076
...,...,...,...,...
1716590,U121730,pageload,2018-05-07 00:06:59.266,pr100166
4183846,U134230,pageload,2018-05-07 00:06:48.110,pr100403
248653,U136956,pageload,2018-05-07 00:03:36.548,pr100312
2580265,U136955,pageload,2018-05-07 00:03:36.548,pr101522


In [34]:
Recently_Viewed_Product=ss[['UserID','ProductID']]

In [35]:
Recently_Viewed_Product.columns=['index','Recently_Viewed_Product']

In [36]:
#val=[]
#for ID in ss['UserID'].unique().tolist():
    #try:
        #temp_dataset=ss[ss['UserID']==ID]
        #val.append(temp_dataset[temp_dataset['visit_date_time']==temp_dataset['visit_date_time'].max()]['ProductID'].values[0])
    #except IndexError:
        #val.append(ss[ss['UserID']==ID]['ProductID'].values[0])

In [37]:
#Recently_Viewed_Product=pd.DataFrame()
#Recently_Viewed_Product['index']=ss['UserID'].unique().tolist()

#Recently_Viewed_Product['Recently_Viewed_Product']=val

In [38]:
df=df.merge(Recently_Viewed_Product,on='index',how='left')

In [39]:
df['Recently_Viewed_Product']=df['Recently_Viewed_Product'].astype('str').replace('nan','Product101')


In [40]:
Test4=data[['UserID','Activity','visit_date_time']]

In [41]:
Test4=data[['UserID','Activity']][data['visit_date_time']>'2018-05-21']

In [42]:
pageload_7_days=Test4[Test4['Activity']=='pageload'].value_counts().to_frame('Pageloads_last_7_days').reset_index(level=1).reset_index(level=0).drop(columns='Activity')

In [43]:
pageload_7_days.columns=['index','Pageloads_last_7_days']

In [44]:
df=df.merge(pageload_7_days,on='index',how='left')

In [45]:
click_7_days=Test4[Test4['Activity']=='click'].value_counts().to_frame('Clicks_last_7_days').reset_index(level=1).reset_index(level=0).drop(columns='Activity')

In [46]:
click_7_days.columns=['index','Clicks_last_7_days']

In [47]:
df=df.merge(click_7_days,on='index',how='left')

In [48]:
df.head()

,index,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
0,U100002,NaN,5.0,5,pr100258,android,pr100258,NaN,NaN
1,U100003,3.0,4.0,4,pr100079,windows,pr100079,1.0,2.0
2,U100004,1.0,28.0,30,pr100753,windows,Product101,1.0,NaN
3,U100005,2.0,5.0,10,pr102072,android,pr100234,1.0,NaN
4,U100006,1.0,1.0,1,pr101111,android,pr101111,1.0,NaN


In [49]:
df.columns=submission.columns.tolist()

In [50]:
df.head()

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
0,U100002,NaN,5.0,5,pr100258,android,pr100258,NaN,NaN
1,U100003,3.0,4.0,4,pr100079,windows,pr100079,1.0,2.0
2,U100004,1.0,28.0,30,pr100753,windows,Product101,1.0,NaN
3,U100005,2.0,5.0,10,pr102072,android,pr100234,1.0,NaN
4,U100006,1.0,1.0,1,pr101111,android,pr101111,1.0,NaN


In [51]:
df['No_of_days_Visited_7_Days']=df['No_of_days_Visited_7_Days'].fillna(0).astype('int')
df['No_Of_Products_Viewed_15_Days']=df['No_Of_Products_Viewed_15_Days'].fillna(0).astype('int')

df['Pageloads_last_7_days']=df['Pageloads_last_7_days'].fillna(0).astype('int')
df['Clicks_last_7_days']=df['Clicks_last_7_days'].fillna(0).astype('int')

In [52]:
df.to_csv("submission_file_4.csv",index=False)